In [ ]:
# Dependencies
import pandas as pd
from pprint import pprint 
import time
import requests as req
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager 

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# Scraping

## NASA Mars News

- Scraped the [NASA Mars News Site](https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest) and collected the latest News Title and Paragraph Text.  
- Assign the text to variables that we can reference later.

In [ ]:
# set up url for NASA news site, browser, and html
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)
news_html = browser.html

# parse with BeautifulSoup
news_soup = bs(news_html, 'html.parser')

# find the latest news article
latest_article = news_soup.find('li', class_='slide')

# find the latest news article title and print it
article_title = latest_article.find('div', class_='content_title').find('a').text
time.sleep(2) # sleep before next task 

# find the latest news article paragraph text and print it
article_p = latest_article.find('div', class_='article_teaser_body').text
time.sleep(2) # sleep before next task 

# print the title and the paragraph text
print(article_title)
print(article_p)
time.sleep(2) # sleep before next task 

## JPL Mars Space Images - Featured Image

- Visit the url for JPL Featured Space Image [here](https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html).
- Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
- Make sure to find the image url to the full size .jpg image.

In [ ]:
# set up url for JPL Featured Mars Image, browser, and html
image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(image_url)
image_html = browser.html
time.sleep(2) # sleep before next task 

# go to 'FULL IMAGE', set the new browser link, and html
browser.links.find_by_partial_text('FULL IMAGE').first.click()
full_image_html = browser.html
time.sleep(2) # sleep before next task 

# parse with BeautifulSoup
image_soup = bs(full_image_html, 'html.parser')
time.sleep(2) # sleep before next task 

# scrape the URL
feature_url = image_soup.find('img', class_='fancybox-image')['src']
time.sleep(2) # sleep before next task 

# print the url for the full image version of the Featured Mars Image
base_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
featured_image_url = f'{base_url}{feature_url}'
print(featured_image_url)
time.sleep(2) # sleep before next task 


## Mars Facts
- Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
- Use Pandas to convert the data to an HTML table string.

In [ ]:
# set up url for Mars Facts and browser
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)
time.sleep(2) # sleep before next task 

# use Pandas to parse facts url to find all tables
facts_tables = pd.read_html(facts_url)

# select the correct table from the list of tables
facts_df = facts_tables[0]

# rename the columns with appropriate headings
facts_df.columns = ['Variable', 'Value']

# convert the data to an HTML string
facts_string = facts_df.to_html(index=False)
print(facts_string)

## Mars Hemispheres
- Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.
- Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
- Append the dictionary with the image url string and the hemisphere title to a list. This list contains one dictionary for each hemisphere.

In [ ]:
# set up url for USGS Astrogeology, browser, and html
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html_hemispheres = browser.html
time.sleep(2) # sleep before next task 

# parse with BeautifulSoup
hemispheres_soup = bs(html_hemispheres, 'html.parser')
time.sleep(2) # sleep before next task 

# retrieve the all items (contains all info for hemispheres)
hemispheres_items = hemispheres_soup.find_all('div', class_='item')

# empty list for titles and image urls
hemispheres_data = []

# set up base url
hemispheres_base_url = 'https://astrogeology.usgs.gov'

# loop through items containing all the hemispheres' info
for item in hemispheres_items: 
    
    # find and store titles
    title = item.find('h3').text
    
    # find link that leads to full res image
    hemispheres_full = item.find('a', class_='itemLink product-item')['href']
    time.sleep(2) # sleep before next task 
    
    # navigate to site containing full res image
    browser.visit(hemispheres_base_url + hemispheres_full)
    time.sleep(2) # sleep before next task 
    
    # save the html of site containing each hemisphere's full res image
    full_res_url = browser.html
    
    # parse each hemisphere's html with BeautifulSoup
    hemispheres_soup = bs(full_res_url, 'html.parser')
    time.sleep(2) # sleep before next task 
    
    # save the path to full res image
    full_res_path = hemispheres_soup.find('img', class_='wide-image')['src']
    
    # save the url to the full res image 
    img_url = f'{hemispheres_base_url}{full_res_path}'
    
    # append title and full res image url to hemispheres dictionary
    hemispheres_data.append({'Title' : title, 'Link to article' : img_url})  
    
# print the hemispheres dictionary
pprint(hemispheres_data)

# close the browser
browser.quit()